In [1]:
import numpy as np
import matplotlib.pyplot as plt
from propagation_functions import *
from cauer import *
import control
%matplotlib qt

# set journal publication figure parameters
import matplotlib as mpl
mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['figure.titlesize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['lines.linewidth'] = 1

# Let's compare water diccusion using diccerent bin sizes. 

fn = "fit.popc.select.dat"
st, end = 11, 88
v_single, d_single, drad_single, edges_single = just_get_me_my_F_and_D(fn,
                                                                       st,
                                                                       end)
dz = (edges_single[-1] - edges_single[0])/(len(edges_single)-1.)

# aveD : Angstrom^2 / ps
aveD = np.array([(d_single[0]+d_single[-1])/2.] +\
                ((d_single[:-1] + d_single[1:])/2.).tolist())
v_ref = v_single[0]
part = np.exp(-(v_single-v_ref))
dRdx = 1./(part*aveD)  # this used to use aveD, but I get better results with d_single
Rmem = np.sum(dRdx[1:-1])*dz  # ps / A
conc = np.exp(-v_single + v_ref)
Cmem = np.sum(conc[1:-1])*dz  # A

RCdiv2 = Rmem*Cmem/2
print("RCdiv2", RCdiv2)

nbins_mem = len(v_single) - 2
Dwater = d_single[0]

print("Cmem", Cmem)
print("Rmem", Rmem)


def get_coarse_system(dz, Dwater, nbins_water_l, nbins_water_r, nwl,
                      nwr, nmem, nbins_mem, verbose=False):
    """
    Get the rate matrix for a water-myelin-water system with variable bin sizes.
    
    Parameters
    ----------
    dz : float
        The microbin size. Other bin sizes are multiples of this.
    Dwater : float
        The diccusion coeccicient for water.
    nbins_water_l : float
        The number of microbins in one water bin on the left.
    nbins_water_r : float
        The number of microbins in one water bin on the right.
    nwl : int
        The number of water bins on the left.
    nwr : int
        The number of water bins on the right.
    nmem : int
        The number of bilayers in the myelin.
    nbins_mem : int
        The number of microbins in one bilayer.

    Returns
    -------
    A, B, C, D: state-space representation of the system.
    mids : the midpoints of the bins.
    K : the rate matrix.
    
    # A = K without first and last row and column
    # B = [K[1,0], 0, 0, ..., 0, K[-2,-1]] # scratch it, we don't need the 0s
    # U(t) = [V_leftsource(t), V_rightsource(t)]
    # X(t) = [X1(t), X2(t), ..., Xn(t)]

    # X'(t) = AX(t) + BU(t)
    # Y(t) = CX(t) + DU(t)  

    # X(t) : (n, 1)
    # A    : (n, n)
    # B    : (n, 2)
    # U(t) : (2, 1)
    # C    : (n, n)
    # D    : (n, 2)

    Notes
    -----
    Note that the source terms are NOT included in the state space representation,
    but they are included in the rate matrix K. Use wisely.

    """
    hwater_l = dz * nbins_water_l
    hwater_r = dz * nbins_water_r
    hmem = dz * nbins_mem
    
    Cs = [hwater_l]*nwl
    RLs = [hwater_l/2./Dwater]*nwl
    RRs = [hwater_l/2./Dwater]*nwl
    mids = [hwater_l/2. + i*hwater_l for i in range(nwl)]
    if verbose:
        print("Total resistance of left water:", sum(RLs))
        print("Total storage capacity of left water:", sum(Cs))

    Cs += [Cmem]*nmem
    RLs += [Rmem/2.]*nmem 
    RRs += [Rmem/2.]*nmem
    if verbose:
        print("Mem, individual elements:")
        print("RL, RR:", RLs[nwl], RRs[nwl])
        print("C:", Cs[nwl])
    mids += [nwl*hwater_l + hmem/2. + i*hmem for i in range(nmem)]
    if verbose:
        print("Total resistance of myelin:", sum(RLs[nwl:]))
        print("Total storage capacity of myelin:", sum(Cs[nwl:]))

    Cs += [hwater_r]*nwr
    RLs += [hwater_r/2./Dwater]*nwr
    RRs += [hwater_r/2./Dwater]*nwr
    mids += [nwl*hwater_l + nmem*hmem + hwater_r/2. + i*hwater_r for i in range(nwr)]
    if verbose:
        print("Total resistance of right water:", sum(RLs[nwl+nmem:]))
        print("Total storage capacity of right water:", sum(Cs[nwl+nmem:]))

    Cs = np.array(Cs)
    RLs = np.array(RLs)
    RRs = np.array(RRs)
    mids = np.array(mids)

    R_leftsource = 0.
    R_rightsource = 0.

    V_leftsource = 1.
    V_rightsource = 0.

    # Define a rate matrix with the source inside
    # We will add the source terms in there as well, so + 2
    K = np.zeros((len(Cs)+2, len(Cs)+2), dtype=float)
    for i in range(len(Cs)):
        idx = i + 1  # occset for source
        if i == 0:
            K[idx, idx] = -1./Cs[i]*(1./(RLs[i] + R_leftsource) +\
                                    1./(RRs[i] + RLs[i+1]))
            K[idx, idx+1] = 1./Cs[i]*1./(RRs[i] + RLs[i+1])
            K[idx, idx-1] = 1./Cs[i]*1./(RLs[i] + R_leftsource)
        elif i == len(Cs) - 1:
            K[idx, idx] = -1./Cs[i]*(1./(RLs[i] + RRs[i-1]) +\
                                    1./(RRs[i] + R_rightsource))
            K[idx, idx-1] = 1./Cs[i]*1./(RLs[i] + RRs[i-1])
            K[idx, idx+1] = 1./Cs[i]*1./(RRs[i] + R_rightsource)
        else:
            K[idx, idx] = -1./Cs[i]*(1./(RLs[i] + RRs[i-1]) +\
                                    1./(RRs[i] + RLs[i+1]))
            K[idx, idx-1] = 1./Cs[i]*1./(RLs[i] + RRs[i-1])
            K[idx, idx+1] = 1./Cs[i]*1./(RRs[i] + RLs[i+1])

    A = K[1:-1, 1:-1]
    B = np.zeros((len(A), 2), dtype=float)
    B[0, 0] = K[1, 0]
    B[-1, 1] = K[-2, -1]
    C = np.diag(Cs)
    D = np.zeros_like(B)

    return A, B, C, D, mids, K

RCdiv2 62385.4639532927
Cmem 325.58566892676504
Rmem 383.2199627147917


In [2]:
# Things we will have to scan
dRs = [10000, 5000, 2500, 1000]  # Not too sure about this one, it will be slower, okay.
nmems = [100, 1, 10, 25, 50]  # definitely.
fs = [2, 3, 4, 5]  # defiantly
delta_fs = [1, 10, 100]  # exponential tau (maybe sigmoid instead?)
Dwaterfactors = [0.5, 1.]  # defiantly
c_init_rs = [1, 5, 10] # defiantly
c_init_ls = [35] # defiantly

parameters = [
    {
    'dR': dR_el,
    'nmem': nmem_el,
    'f': f_el,
    'delta_f': delta_f_el,
    'Dw_f': Dwaterfactor_el,
    'c_init_l': c_init_l_el,
    'c_init_r': c_init_r_el,
    'nwl': 100,
    'nwr': 20,
    }
for dR_el in dRs
for nmem_el in nmems
for f_el in fs
for delta_f_el in delta_fs
for Dwaterfactor_el in Dwaterfactors
for c_init_l_el in c_init_ls
for c_init_r_el in c_init_rs
]
print(len(parameters))

1440


In [4]:
overwrite = True

# left water layer + myelin should be dz*100*76 + 15um
# right water layer should be 0.5um
dtottarget = dz*(100*76) + 200000 + 10000  # Ang
print("Target total distance:", dtottarget)
#############################################
results = []
for n_par, parameterset in enumerate(parameters):
    # Now we specify dR in the beginning
    # print("Working with parameterset: ")
    #print(parameterset)
    nmem=parameterset['nmem']
    nbins_mem=76

    dR = parameterset['dR']  # Ang
    dM = nmem * nbins_mem *dz # Anghold the effect of the
    dL = dtottarget - dR - dM  # Ang

    nwl = parameterset['nwl']
    nwr = parameterset['nwr']
    nmicrobinsl = dL/dz/nwl
    nmicrobinsr = dR/dz/nwr
    # print("float of microbins left:", nmicrobinsl)
    # print("float of microbins right:", nmicrobinsr)

    dtot = nwl*dz*nmicrobinsl + nwr*dz*nmicrobinsr + dM
    # print("Total distance:", dtot)

    Dwater_factor = parameterset['Dw_f']

    c_init_l = parameterset['c_init_l']
    c_init_r = parameterset['c_init_r']
    ocr_increase = parameterset['f']

    factor = 10**9
    tau_ocr = parameterset['delta_f']*factor


    if overwrite:
        c_init_l = 35
        c_init_r = 25
        ocr_increase = 5
        Dwater_factor = 0.5
        nmem = 100
        tau_ocr = 100

    A1, B1, C1, D1, mids1, K1 = get_coarse_system(dz, Dwater*Dwater_factor,
                                                nmicrobinsl, nmicrobinsr,
                                                nwl, nwr, nmem,
                                                nbins_mem)
    # print("final mids value:", mids1[-1])
    evals, evecs = np.linalg.eig(A1)
    # All eigenvalues must be negative because no sources in matrix
    tau_max = -1./np.max(evals)
    # Constant concentration left and right
    Achosen, Bchosen, Cchosen = A1, B1, C1
    Teq = np.zeros((len(Achosen)+2, len(Achosen)+2), float)
    Teq[1,0] = Bchosen[0,0]
    Teq[-2,-1] = Bchosen[-1,-1]
    for i in range(len(Achosen)):
        for j in range(len(Achosen)):
            Teq[i+1,j+1] = Achosen[i,j]
    # Constant flux left and right
    Tf = Teq.copy()
    Tf[0,0] = - Tf[1,0]
    Tf[-1,-1] = - Tf[-2,-1]
    Tf[0,1] = Tf[1,0]
    Tf[-1,-2] = Tf[-2,-1]
    # constant concentration right, constant flux left
    Tfc = Teq.copy()
    Tfc[0,0] = -Tfc[1,0]
    Tfc[0,1] = Tfc[1,0]

    # Constant flux left and right
    A_ff = Tf
    B_ff = np.zeros((len(A_ff), 2), float)
    B_ff[0,0] = B_ff[-1,-1] = 1.
    C_ff = np.eye(len(A_ff))
    D_ff = np.array([[0,0] for i in range(len(A_ff))])
    IO_FLUX = control.LinearIOSystem(
        control.StateSpace(A_ff, B_ff, C_ff, D_ff),
        inputs=2, outputs=len(A_ff), states=len(A_ff),
        name='constantqs flux left and right'
    )
    statespace_ff = control.StateSpace(A_ff, B_ff, C_ff, D_ff)
    # constant concentration right, constant flux left
    A_fc = Tfc
    B_fc = np.zeros((len(A_fc), 2), float)
    B_fc[0,0] = 1.
    B_fc[-1,-1] = 1.
    C_fc = np.eye(len(A_fc))
    D_fc = np.array([[0,0] for i in range(len(A_fc))])
    IO_FLUX_CONC = control.LinearIOSystem(
        control.StateSpace(A_fc, B_fc, C_fc, D_fc),
        inputs=2, outputs=len(A_fc), states=len(A_fc),
        name='constant flux left, cqsonstant concentration right'
    )
    statespace_fc = control.StateSpace(A_fc, B_fc, C_fc, D_fc)

    

    # Now we switch to the constant flux model (FF)
    # X0_ff = x_out_cc[:,-1]
    # the initial state, X0_ff, is only approximated by the final state of the CC model
    # we went 10 tau_max in the CC model, but that's not infinite time
    # we should have gone to infinity to get the exact initial state for the FF model
    # but we can calculate the exact initial state of the FF model by solving the 
    # steady-state equation for the FF model
    # It is the eigenvector of the FF model with eigenvalue 0 
    evals, evecs = np.linalg.eig(Teq)
    idx = evals.argsort()
    evals = evals[idx]
    evecs = evecs[:,idx]
    X0_ff_l = evecs[:, -2]
    X0_ff_r = evecs[:, -1]
    X0_ff = X0_ff_l/X0_ff_l[0]*c_init_l + X0_ff_r/X0_ff_r[-1]*c_init_r
    # Same thing for the steady-state flux, we can calculate it exactly
    # divisor = (dz*250/2/Dwater*dz*250)/Dwater_factor
    # U_U_ss_flux_U = -1.*(x_out_cc[1][-1]-x_out_cc[0][-1])/divisor
    # ss_flux_U = np.mean([-(X0_ff[1] - X0_ff[0])*B1[0,0],
    #                      -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]])
    #                      #weights = [C1[1,1], C1[-1,-1]])
    # print(f"SS-flux: {ss_flux_U * C1[1,1]}")

    ss_flux_lU = -(X0_ff[1] - X0_ff[0])*B1[0,0]
    ss_flux_rU = -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]
    # print("ss_flux:", ss_flux_lU*C1[1,1], ss_flux_rU*C1[-1,-1])
    # print("ss_flux_lU:", ss_flux_lU, "ss_flux_rU:", ss_flux_rU)
    parameterset['ss_flux_lU'] = ss_flux_lU
    parameterset['ss_flux_rU'] = ss_flux_rU
    parameterset['ss_flux_l'] = ss_flux_lU*C1[1,1]
    parameterset['ss_flux_r'] = ss_flux_lU*C1[-1,-1]
    # Below is the one in mol / m^3 / ps * A
    #print(1 / (nmem * Rmem + (nwl + nwr) * nmicrobins * dz / (Dwater*Dwater_factor) ) * (X0_ff[0] - X0_ff[-1]))

    # Now we increase the OCR by 300% on the right, and see what happens
    tau_max_screen = tau_max*10+tau_ocr*5
    t_ocr = np.linspace(0, tau_max_screen, 1000)
    X0_ocr = X0_ff
    Ul_ocr = np.ones_like(t_ocr)*ss_flux_lU
    Ur_ocr = -1*np.ones_like(t_ocr)*ss_flux_rU
    Ur_ocr -= np.array([(1.-np.exp(-t/tau_ocr)) for t in t_ocr])*ss_flux_rU*(ocr_increase-1.)

    U_ocr = np.vstack((Ul_ocr,Ur_ocr))

    t_out_ocr, y_out_ocr, x_out_ocr = control.forced_response(
        statespace_ff, T=t_ocr, U=U_ocr, X0=X0_ocr, return_x=True
    )

    # Find the index of the zero-concentration bin
    nill_index = None 
    for i in range(len(y_out_ocr[-1])):
        if y_out_ocr[-1][i] < 0:
            nill_index = i
            break
    assert nill_index is not None
    if y_out_ocr[-1][nill_index] < 0:
        nill_index -= 1
    print("nill_index:", nill_index, "time", t_out_ocr[nill_index]/factor, "ms")
    while nill_index < 500:
        # refine the search
        t_ocr = np.linspace(t_ocr[nill_index-1], t_ocr[nill_index+1], 1000)
        X0_ocr = x_out_ocr[:, nill_index-1]
        Ul_ocr = np.ones_like(t_ocr)*ss_flux_lU
        Ur_ocr = -1*np.ones_like(t_ocr)*ss_flux_rU
        Ur_ocr -= np.array([(1.-np.exp(-t/tau_ocr)) for t in t_ocr])*ss_flux_rU*(ocr_increase-1.)
        U_ocr = np.vstack((Ul_ocr,Ur_ocr))

        t_out_ocr, y_out_ocr, x_out_ocr = control.forced_response(
            statespace_ff, T=t_ocr, U=U_ocr, X0=X0_ocr, return_x=True
        )

        # Find the index of the zero-concentration bin
        nill_index = None 
        for i in range(len(y_out_ocr[-1])):
            if y_out_ocr[-1][i] < 0:
                nill_index = i
                break
        assert nill_index is not None
        if y_out_ocr[-1][nill_index] < 0:
            nill_index -= 1
        print(f"nill_index: {nill_index}, time: {t_out_ocr[nill_index]/factor} ms")
    parameterset['nill_index'] = nill_index
    parameterset['t_nill'] = t_out_ocr[nill_index]/factor

    print(f"Done with paramterset {n_par}")
    ohlenoes
    results.append(parameterset.copy())
    parameterset = {}

Target total distance: 215162.3487179487
nill_index: 377 time 73.4020650894786 ms
nill_index: 765, time: 73.50555451142709 ms
Done with paramterset 0


NameError: name 'ohlenoes' is not defined

In [5]:
fig, ax = plt.subplots()
ax.plot([0],[-(X0_ff[1] - X0_ff[0])*B1[0,0]], "-x")
ax.plot(-(x_out_ocr[1] - x_out_ocr[0])*B1[0,0])
fig.show()


In [24]:
# let's plot the final concentration profile
fig, ax = plt.subplots()
ax.pllot()

(172, 1000)

In [10]:
mmHg = 0.044025157232704404 #mg/L
mgdivL = 1/mmHg  # mmHg
# we gave concentration in mmHg, so we need to convert it to mol/L
# 1 mg/L = 1/32 mol/m^3
# 1 mmHg = 0.044025157232704404/32 mol/m^3
conv = 0.044025157232704404/32
conv_time = conv * 10**-10 / 10**-12  # will give flux in mol/m^2/s
conv_time2 = conv_time * 10**6  # will give flux in micromol/m^2/s

#
markers = ['o', 's', 'd', '^', 'v', '<', '>', 'p', 'h', 'D']
colors = ['b', 'g', 'r', 'purple', 'm', 'y', 'k', 'w']
xvals = [5, 1, 2, 3, 4]
xvals2 = [-0.1, +0.1]
xval_big = [0, 5]

fig, axs = plt.subplots(4, 3, sharex='col', figsize=(15, 10))

for i, chosen_cir in enumerate([1, 5, 10]):
    for j, chosen_df in enumerate([1, 10, 100]):
        ax = axs[j+1, i]
        for result in results:
            
            color = colors[fs.index(result['f'])]
            marker = markers[dRs.index(result['dR'])]
            xv = xvals[nmems.index(result['nmem'])]
            #xv += xvals2[c_init_rs.index(result['c_init_r'])]
            xv += xval_big[Dwaterfactors.index(result['Dw_f'])]
            yv = result['t_nill']
            yv2 = result['ss_flux_l']*conv_time2
            # chosen_cir = 10
            # chosen_df = 1
            if result['c_init_r'] == chosen_cir and result['delta_f'] == chosen_df:
                ax.plot(xv, yv, marker=marker, color=color)

                if j==0:
                    axs[0,i].plot(xv, yv2, marker=marker, color="k")
                    axs[0,i].tick_params(axis='both', which='major', labelsize=17)
                    axs[0,i].tick_params(axis='both', which='minor', labelsize=17)
                    axs[0,i].axvline(x=5.5, color='k')
                    axs[0,i].set_ylim(3,12)
                    axs[0,i].yaxis.set_major_locator(plt.MaxNLocator(5))
                    if i==0:
                        axs[j,i].set_ylabel(r"$J^{\mathrm{ss}}$ [$\mu$mol$\,m^{-2}\,s^{-1}$]", fontsize=17)

        ax.axvline(x=5.5, color='k')
        if j < 2:
            ax.set_xticklabels([])
        else:
            ax.set_xticks(xvals+[xval + xval_big[1] for xval in xvals])
            ax.set_xticklabels(nmems + nmems)
            ax.set_xlabel(r"Number of bilayers $M$", fontsize=17)
        
        if i == 0:
            ax.set_ylabel(r"$T$ [ms]", fontsize=17)
        # labelfontsize = 15, tickmarksize = 15
        # force 5 ticks on y-axis, not min not max, but 5
        ax.yaxis.set_major_locator(plt.MaxNLocator(5))

        ax.tick_params(axis='both', which='major', labelsize=17)
        ax.tick_params(axis='both', which='minor', labelsize=17)


plt.tight_layout()
fig.show()
fig.savefig('newdepletions.pdf')



In [1]:
# get the min and max steady-state fluxes
min_flux = np.inf
max_flux = -np.inf
for result in results:
    if result['ss_flux_l']*conv_time2 < min_flux:
        min_flux = result['ss_flux_l']*conv_time2
    if result['ss_flux_l']*conv_time2 > max_flux:
        max_flux = result['ss_flux_l']*conv_time2

print("min_flux:", min_flux)
print("max_flux:", max_flux)

NameError: name 'np' is not defined

In [ ]:
# left water layer + myelin should be dz*100*76 + 15um
# right water layer should be 0.5um
dtottarget = dz*(100*76) + 200000 + 5000  # Ang
print("Target total distance:", dtottarget)
#############################################
results = []
for n_par, parameterset in enumerate(parameters):
    # Now we specify dR in the beginning
    # print("Working with parameterset: ")
    #print(parameterset)
    nmem=parameterset['nmem']
    nbins_mem=76

    dR = parameterset['dR']  # Ang
    dM = nmem * nbins_mem *dz # Ang
    dL = dtottarget - dR - dM  # Ang

    nwl = parameterset['nwl']
    nwr = parameterset['nwr']
    nmicrobinsl = dL/dz/nwl
    nmicrobinsr = dR/dz/nwr
    # print("float of microbins left:", nmicrobinsl)
    # print("float of microbins right:", nmicrobinsr)

    dtot = nwl*dz*nmicrobinsl + nwr*dz*nmicrobinsr + dM
    # print("Total distance:", dtot)

    Dwater_factor = parameterset['Dw_f']

    c_init_l = parameterset['c_init_l']
    c_init_r = parameterset['c_init_r']
    ocr_increase = parameterset['f']

    factor = 10**9


    A1, B1, C1, D1, mids1, K1 = get_coarse_system(dz, Dwater*Dwater_factor,
                                                nmicrobinsl, nmicrobinsr,
                                                nwl, nwr, nmem,
                                                nbins_mem)
    # print("final mids value:", mids1[-1])
    evals, evecs = np.linalg.eig(A1)
    # All eigenvalues must be negative because no sources in matrix
    tau_max = -1./np.max(evals)
    # Constant concentration left and right
    Achosen, Bchosen, Cchosen = A1, B1, C1
    Teq = np.zeros((len(Achosen)+2, len(Achosen)+2), float)
    Teq[1,0] = Bchosen[0,0]
    Teq[-2,-1] = Bchosen[-1,-1]
    for i in range(len(Achosen)):
        for j in range(len(Achosen)):
            Teq[i+1,j+1] = Achosen[i,j]
    # Constant flux left and right
    Tf = Teq.copy()
    Tf[0,0] = - Tf[1,0]
    Tf[-1,-1] = - Tf[-2,-1]
    Tf[0,1] = Tf[1,0]
    Tf[-1,-2] = Tf[-2,-1]
    # constant concentration right, constant flux left
    Tfc = Teq.copy()
    Tfc[0,0] = -Tfc[1,0]
    Tfc[0,1] = Tfc[1,0]

    # Constant flux left and right
    A_ff = Tf
    B_ff = np.zeros((len(A_ff), 2), float)
    B_ff[0,0] = B_ff[-1,-1] = 1.
    C_ff = np.eye(len(A_ff))
    D_ff = np.array([[0,0] for i in range(len(A_ff))])
    IO_FLUX = control.LinearIOSystem(
        control.StateSpace(A_ff, B_ff, C_ff, D_ff),
        inputs=2, outputs=len(A_ff), states=len(A_ff),
        name='constantqs flux left and right'
    )
    statespace_ff = control.StateSpace(A_ff, B_ff, C_ff, D_ff)
    # constant concentration right, constant flux left
    A_fc = Tfc
    B_fc = np.zeros((len(A_fc), 2), float)
    B_fc[0,0] = 1.
    B_fc[-1,-1] = 1.
    C_fc = np.eye(len(A_fc))
    D_fc = np.array([[0,0] for i in range(len(A_fc))])
    IO_FLUX_CONC = control.LinearIOSystem(
        control.StateSpace(A_fc, B_fc, C_fc, D_fc),
        inputs=2, outputs=len(A_fc), states=len(A_fc),
        name='constant flux left, cqsonstant concentration right'
    )
    statespace_fc = control.StateSpace(A_fc, B_fc, C_fc, D_fc)

    # Now we switch to the constant flux model (FF)
    # X0_ff = x_out_cc[:,-1]
    # the initial state, X0_ff, is only approximated by the final state of the CC model
    # we went 10 tau_max in the CC model, but that's not infinite time
    # we should have gone to infinity to get the exact initial state for the FF model
    # but we can calculate the exact initial state of the FF model by solving the 
    # steady-state equation for the FF model
    # It is the eigenvector of the FF model with eigenvalue 0 
    evals, evecs = np.linalg.eig(Teq)
    idx = evals.argsort()
    evals = evals[idx]
    evecs = evecs[:,idx]
    X0_ff_l = evecs[:, -2]
    X0_ff_r = evecs[:, -1]
    X0_ff = X0_ff_l/X0_ff_l[0]*c_init_l + X0_ff_r/X0_ff_r[-1]*c_init_r
    # Same thing for the steady-state flux, we can calculate it exactly
    # divisor = (dz*250/2/Dwater*dz*250)/Dwater_factor
    # U_U_ss_flux_U = -1.*(x_out_cc[1][-1]-x_out_cc[0][-1])/divisor
    # ss_flux_U = np.mean([-(X0_ff[1] - X0_ff[0])*B1[0,0],
    #                      -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]])
    #                      #weights = [C1[1,1], C1[-1,-1]])
    # print(f"SS-flux: {ss_flux_U * C1[1,1]}")

    ss_flux_lU = -(X0_ff[1] - X0_ff[0])*B1[0,0]
    ss_flux_rU = -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]
    # print("ss_flux:", ss_flux_lU*C1[1,1], ss_flux_rU*C1[-1,-1])
    # print("ss_flux_lU:", ss_flux_lU, "ss_flux_rU:", ss_flux_rU)
    parameterset['ss_flux_lU'] = ss_flux_lU
    parameterset['ss_flux_rU'] = ss_flux_rU
    parameterset['ss_flux_l'] = ss_flux_lU*C1[1,1]
    parameterset['ss_flux_r'] = ss_flux_lU*C1[-1,-1]
    # Below is the one in mol / m^3 / ps * A
    #print(1 / (nmem * Rmem + (nwl + nwr) * nmicrobins * dz / (Dwater*Dwater_factor) ) * (X0_ff[0] - X0_ff[-1]))

    # Now we increase the OCR by 300% on the right, and see what happens
    tau_max_screen = tau_max*100
    t_ocr = np.linspace(0, tau_max_screen, 10000)
    X0_ocr = X0_ff
    Ul_ocr = np.ones_like(t_ocr)*ss_flux_lU
    Ur_ocr = -1*np.ones_like(t_ocr)*ss_flux_rU*ocr_increase
    U_ocr = np.vstack((Ul_ocr,Ur_ocr))

    t_out_ocr, y_out_ocr, x_out_ocr = control.forced_response(
        statespace_ff, T=t_ocr, U=U_ocr, X0=X0_ocr, return_x=True
    )

    # Find the index of the zero-concentration bin
    nill_index = None 
    for i in range(len(y_out_ocr[-1])):
        if y_out_ocr[-1][i] < 0:
            nill_index = i
            break
    assert nill_index is not None
    if y_out_ocr[-1][nill_index] < 0:
        nill_index -= 1
    print("nill_index:", nill_index, "time", t_out_ocr[nill_index]/factor, "ms")
    while nill_index < 5000:
        # refine the search
        t_ocr = np.linspace(t_ocr[nill_index-1], t_ocr[nill_index+1], 10000)
        X0_ocr = x_out_ocr[:, nill_index-1]
        Ul_ocr = np.ones_like(t_ocr)*ss_flux_lU
        Ur_ocr = -1*np.ones_like(t_ocr)*ss_flux_rU*ocr_increase
        U_ocr = np.vstack((Ul_ocr,Ur_ocr))

        t_out_ocr, y_out_ocr, x_out_ocr = control.forced_response(
            statespace_ff, T=t_ocr, U=U_ocr, X0=X0_ocr, return_x=True
        )

        # Find the index of the zero-concentration bin
        nill_index = None 
        for i in range(len(y_out_ocr[-1])):
            if y_out_ocr[-1][i] < 0:
                nill_index = i
                break
        assert nill_index is not None
        if y_out_ocr[-1][nill_index] < 0:
            nill_index -= 1
        print(f"nill_index: {nill_index}, time: {t_out_ocr[nill_index]/factor} ms")
    parameterset['nill_index'] = nill_index
    parameterset['t_nill'] = t_out_ocr[nill_index]/factor

    print(f"Done with paramterset {n_par}")
    results.append(parameterset.copy())
    parameterset = {}

In [4]:
# left water layer + myelin should be dz*100*76 + 15um
# right water layer should be 0.5um
dtottarget = dz*(100*76) + 200000 + 5000  # Ang
print("Target total distance:", dtottarget)
#############################################

# Now we specify dR in the beginning
nmem=100
nbins_mem=76

dR = 5000  # Ang
dM = nmem * nbins_mem *dz # Ang
dL = dtottarget - dR - dM  # Ang

nwl = 100
nwr = 30
nmicrobinsl = dL/dz/nwl
nmicrobinsr = dR/dz/nwr
print("Number of microbins left:", nmicrobinsl)
print("Number of microbins right:", nmicrobinsr)

dtot = nwl*dz*nmicrobinsl + nwr*dz*nmicrobinsr + dM
print("Total distance:", dtot)

Dwater_factor = 0.5

c_init_l=35.
c_init_r=10.
ocr_increase=2

A1, B1, C1, D1, mids1, K1 = get_coarse_system(dz, Dwater*Dwater_factor,
                                              nmicrobinsl, nmicrobinsr,
                                              nwl, nwr, nmem,
                                              nbins_mem)
print("final mids value:", mids1[-1])
evals, evecs = np.linalg.eig(A1)
# All eigenvalues must be negative because no sources in matrix
tau_max = -1./np.max(evals)
# Constant concentration left and right
Achosen, Bchosen, Cchosen = A1, B1, C1
Teq = np.zeros((len(Achosen)+2, len(Achosen)+2), float)
Teq[1,0] = Bchosen[0,0]
Teq[-2,-1] = Bchosen[-1,-1]
for i in range(len(Achosen)):
    for j in range(len(Achosen)):
        Teq[i+1,j+1] = Achosen[i,j]
# Constant flux left and right
Tf = Teq.copy()
Tf[0,0] = - Tf[1,0]
Tf[-1,-1] = - Tf[-2,-1]
Tf[0,1] = Tf[1,0]
Tf[-1,-2] = Tf[-2,-1]
# constant concentration right, constant flux left
Tfc = Teq.copy()
Tfc[0,0] = -Tfc[1,0]
Tfc[0,1] = Tfc[1,0]

# Constant flux left and right
A_ff = Tf
B_ff = np.zeros((len(A_ff), 2), float)
B_ff[0,0] = B_ff[-1,-1] = 1.
C_ff = np.eye(len(A_ff))
D_ff = np.array([[0,0] for i in range(len(A_ff))])
IO_FLUX = control.LinearIOSystem(
    control.StateSpace(A_ff, B_ff, C_ff, D_ff),
    inputs=2, outputs=len(A_ff), states=len(A_ff),
    name='constantqs flux left and right'
)
statespace_ff = control.StateSpace(A_ff, B_ff, C_ff, D_ff)
# constant concentration right, constant flux left
A_fc = Tfc
B_fc = np.zeros((len(A_fc), 2), float)
B_fc[0,0] = 1.
B_fc[-1,-1] = 1.
C_fc = np.eye(len(A_fc))
D_fc = np.array([[0,0] for i in range(len(A_fc))])
IO_FLUX_CONC = control.LinearIOSystem(
    control.StateSpace(A_fc, B_fc, C_fc, D_fc),
    inputs=2, outputs=len(A_fc), states=len(A_fc),
    name='constant flux left, cqsonstant concentration right'
)
statespace_fc = control.StateSpace(A_fc, B_fc, C_fc, D_fc)

# Now we switch to the constant flux model (FF)
# X0_ff = x_out_cc[:,-1]
# the initial state, X0_ff, is only approximated by the final state of the CC model
# we went 10 tau_max in the CC model, but that's not infinite time
# we should have gone to infinity to get the exact initial state for the FF model
# but we can calculate the exact initial state of the FF model by solving the 
# steady-state equation for the FF model
# It is the eigenvector of the FF model with eigenvalue 0 
evals, evecs = np.linalg.eig(Teq)
idx = evals.argsort()
evals = evals[idx]
evecs = evecs[:,idx]
X0_ff_l = evecs[:, -2]
X0_ff_r = evecs[:, -1]
X0_ff = X0_ff_l/X0_ff_l[0]*c_init_l + X0_ff_r/X0_ff_r[-1]*c_init_r
# Same thing for the steady-state flux, we can calculate it exactly
# divisor = (dz*250/2/Dwater*dz*250)/Dwater_factor
# U_U_ss_flux_U = -1.*(x_out_cc[1][-1]-x_out_cc[0][-1])/divisor
# ss_flux_U = np.mean([-(X0_ff[1] - X0_ff[0])*B1[0,0],
#                      -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]])
#                      #weights = [C1[1,1], C1[-1,-1]])
# print(f"SS-flux: {ss_flux_U * C1[1,1]}")

ss_flux_lU = -(X0_ff[1] - X0_ff[0])*B1[0,0]
ss_flux_rU = -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]
print(f"ss_flux: {ss_flux_lU*C1[1,1]}")
print(f"ss_flux: {ss_flux_rU*C1[-1,-1]}")
print("FLUX = ", ss_flux_lU*C1[1,1] / 30 * 0.045 * 10**-10 / 10**-12, "mol/m^2/s")
# Below is the one in mol / m^3 / ps * A
#print(1 / (nmem * Rmem + (nwl + nwr) * nmicrobins * dz / (Dwater*Dwater_factor) ) * (X0_ff[0] - X0_ff[-1]))

# Now we increase the OCR by 300% on the right, and see what happens
print("taumax:", tau_max)
tau_max_screen = tau_max/10 if ocr_increase > 4.9 else tau_max
t_ocr = np.linspace(0, tau_max_screen, 10000)
X0_ocr = X0_ff
Ul_ocr = np.ones_like(t_ocr)*ss_flux_lU
Ur_ocr = -1*np.ones_like(t_ocr)*ss_flux_rU*ocr_increase
U_ocr = np.vstack((Ul_ocr,Ur_ocr))

t_out_ocr, y_out_ocr, x_out_ocr = control.forced_response(
    statespace_ff, T=t_ocr, U=U_ocr, X0=X0_ocr, return_x=True
)

# Find the index of the zero-concentration bin
nill_index = None 
for i in range(len(y_out_ocr[-1])):
    if y_out_ocr[-1][i] < 0:
        nill_index = i
        break
assert nill_index is not None
if y_out_ocr[-1][nill_index] < 0:
    nill_index -= 1
factor = 10**9
print("time to reach zero concentration:", t_out_ocr[nill_index]/factor, 'ms')
print("At nillindex", nill_index)
initial_state = x_out_ocr[:,nill_index]
initial_state[-1] = 0.
# and propagate with constant concentration right (and constant influx left)
# but we do this for longer (2 seconds, before CBF response kicks in)
tnew = np.linspace(0, tau_max, 10000)
Ul_fc = np.ones_like(tnew)*ss_flux_lU
Ur_fc = np.zeros_like(tnew)
U_fc = np.vstack((Ul_fc,Ur_fc))

t_out_fc, y_out_fc, x_out_fc = control.forced_response(
    statespace_fc, T=tnew, U=U_fc, X0=initial_state, return_x=True
)

t3 = t_out_ocr[:nill_index]/factor
t4 = t3[-1] + t_out_fc/factor
fig, ax = plt.subplots()
for i in range(1,len(x_out_fc)-1):
    ax.plot(t3, x_out_ocr[i][:nill_index]*Cchosen[i-1,i-1], '-')
    ax.plot(t4, x_out_fc[i]*Cchosen[i-1,i-1], '-')
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Oxygen concentration (A)")
fig.show()

# we calculate the amount of oxygen consumed after the OCR increase
# by integrating the flux over time
# the flux is constant, so we can just multiply
# the flux by the time
# we integrate from the time the OCR increase starts
# to the time it ends

# consumed = 
oxygens = [0.]
time_increase = [0.]
flux_increase = ss_flux_rU*ocr_increase
for i in range(1, nill_index):
    oxygens.append(oxygens[-1] + flux_increase*(t_out_ocr[i]-t_out_ocr[i-1]))
    time_increase.append(time_increase[-1] + (t_out_ocr[i]-t_out_ocr[i-1]))

# herafter, we set the right side at zero concentration, so we can 
# calculate the amount of oxygen that is consumed by 
# doing 
for i in range(1, len(t_out_fc)):
    flux = -1.*(x_out_fc[-1][i]-x_out_fc[-2][i])*B1[0,0]
    oxygens.append(oxygens[-1] + flux*(t_out_fc[i]-t_out_fc[i-1]))
    time_increase.append(time_increase[-1] + (t_out_fc[i]-t_out_fc[i-1]))

dt = t_out_fc[2]-t_out_fc[1]
fig, ax = plt.subplots()
ax.plot(time_increase, oxygens, '-x', label="1mem")
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Oxygen consumed")
ax.legend()
fig.show()

flux_fc = np.array([-A_fc[i, i+1]*x_out_fc[i+1,:]*C1[i+1,i+1] + A_fc[i+1, i]*x_out_fc[i,:]*C1[i+1,i+1] for i in range(1,len(A_fc)-3)])
flux_ocr = np.array([-A_ff[i, i+1]*x_out_ocr[i+1,:]*C1[i+1,i+1] + A_ff[i+1, i]*x_out_ocr[i,:]*C1[i+1,i+1] for i in range(1,len(A_ff)-3)])


fig, ax = plt.subplots()

for i in range(1,len(flux_fc)):
    ax.plot(t3, flux_ocr[i][:nill_index], '-')
    ax.plot(t4, flux_fc[i], '-')
fig.show()

Target total distance: 210162.3487179487
Number of microbins left: 2944.396210033596
Number of microbins right: 245.36635083613302
Total distance: 210162.3487179487
final mids value: 210079.0153846154
ss_flux: 2.957983496819778e-05
ss_flux: 2.9579834967861984e-05
FLUX =  4.436975245229666e-06 mol/m^2/s
taumax: 18480726158.8583


AssertionError: 

In [ ]:
print(ss_flux * 10**-10 / 10**-12, "mmHg*m/s")
print("-----")
print(ss_flux / 30 * 0.045 * 10**-10 / 10**-12, "mol/m^2/s")
print("-----")
print(ss_flux / 30 * 0.045 * 10**-10 / 10**-12 / 10**9, "mol/(um * mm)/s")
N_avo = 6.02214076 * 10**23
print(ss_flux / 30 * 0.045 * 10**-10 / 10**-12 / 10**9 * N_avo, "#O2/(um * mm)/s")

In [4]:
# Now we change the OCR value exponentially towards ocr_increase, with 
# a time constant of tau_ocr (we'll take 1 ms, then 10 ms, then 100 ms)
# and we'll see how the oxygen concentration changes

fige, axe = plt.subplots()
# left water layer + myelin should be dz*100*76 + 15um
# right water layer should be 0.5um
dLM = dz*(100*76) + 190000  # Ang
dR = 5000  # Ang
dtottarget = dLM + dR  # Ang
print("Target total distance:", dtottarget)
#############################################

# Now we specify dR in the beginning
nmem=100
nbins_mem=76

dR = 5000  # Ang
dM = nmem * nbins_mem *dz # Ang
dL = dtottarget - dR - dM  # Ang

nwl = 100
nwr = 30
nmicrobinsl = dL/dz/nwl
nmicrobinsr = dR/dz/nwr
print("Number of microbins left:", nmicrobinsl)
print("Number of microbins right:", nmicrobinsr)

dtot = nwl*dz*nmicrobinsl + nwr*dz*nmicrobinsr + dM
print("Total distance:", dtot)

Dwater_factor = .5
factor = 10**9  # conversion from ps to ms
tau_ocr = 100. * factor
# we simulate up to t_max ms
tau_max_screen = tau_ocr * 5

c_init_l=35.
c_init_r=10.
ocr_increase=2

A1, B1, C1, D1, mids1, K1 = get_coarse_system(dz, Dwater*Dwater_factor,
                                              nmicrobinsl, nmicrobinsr,
                                              nwl, nwr, nmem,
                                              nbins_mem)
print("final mids value:", mids1[-1])
evals, evecs = np.linalg.eig(A1)
# All eigenvalues must be negative because no sources in matrix
tau_max = -1./np.max(evals)
# Constant concentration left and right
Achosen, Bchosen, Cchosen = A1, B1, C1
Teq = np.zeros((len(Achosen)+2, len(Achosen)+2), float)
Teq[1,0] = Bchosen[0,0]
Teq[-2,-1] = Bchosen[-1,-1]
for i in range(len(Achosen)):
    for j in range(len(Achosen)):
        Teq[i+1,j+1] = Achosen[i,j]
# Constant flux left and right
Tf = Teq.copy()
Tf[0,0] = - Tf[1,0]
Tf[-1,-1] = - Tf[-2,-1]
Tf[0,1] = Tf[1,0]
Tf[-1,-2] = Tf[-2,-1]
# constant concentration right, constant flux left
Tfc = Teq.copy()
Tfc[0,0] = -Tfc[1,0]
Tfc[0,1] = Tfc[1,0]

# Constant flux left and right
A_ff = Tf
B_ff = np.zeros((len(A_ff), 2), float)
B_ff[0,0] = B_ff[-1,-1] = 1.
C_ff = np.eye(len(A_ff))
D_ff = np.array([[0,0] for i in range(len(A_ff))])
IO_FLUX = control.LinearIOSystem(
    control.StateSpace(A_ff, B_ff, C_ff, D_ff),
    inputs=2, outputs=len(A_ff), states=len(A_ff),
    name='constantqs flux left and right'
)
statespace_ff = control.StateSpace(A_ff, B_ff, C_ff, D_ff)
# constant concentration right, constant flux left
A_fc = Tfc
B_fc = np.zeros((len(A_fc), 2), float)
B_fc[0,0] = 1.
B_fc[-1,-1] = 1.
C_fc = np.eye(len(A_fc))
D_fc = np.array([[0,0] for i in range(len(A_fc))])
IO_FLUX_CONC = control.LinearIOSystem(
    control.StateSpace(A_fc, B_fc, C_fc, D_fc),
    inputs=2, outputs=len(A_fc), states=len(A_fc),
    name='constant flux left, cqsonstant concentration right'
)
statespace_fc = control.StateSpace(A_fc, B_fc, C_fc, D_fc)

# Now we switch to the constant flux model (FF)
# X0_ff = x_out_cc[:,-1]
# the initial state, X0_ff, is only approximated by the final state of the CC model
# we went 10 tau_max in the CC model, but that's not infinite time
# we should have gone to infinity to get the exact initial state for the FF model
# but we can calculate the exact initial state of the FF model by solving the 
# steady-state equation for the FF model
# It is the eigenvector of the FF model with eigenvalue 0 
evals, evecs = np.linalg.eig(Teq)
idx = evals.argsort()
evals = evals[idx]
evecs = evecs[:,idx]
X0_ff_l = evecs[:, -2]
X0_ff_r = evecs[:, -1]
X0_ff = X0_ff_l/X0_ff_l[0]*c_init_l + X0_ff_r/X0_ff_r[-1]*c_init_r
# Same thing for the steady-state flux, we can calculate it exactly
# divisor = (dz*250/2/Dwater*dz*250)/Dwater_factor
# U_U_ss_flux_U = -1.*(x_out_cc[1][-1]-x_out_cc[0][-1])/divisor
# ss_flux_U = np.mean([-(X0_ff[1] - X0_ff[0])*B1[0,0],
#                      -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]])
#                      #weights = [C1[1,1], C1[-1,-1]])
# print(f"SS-flux: {ss_flux_U * C1[1,1]}")

ss_flux_lU = -(X0_ff[1] - X0_ff[0])*B1[0,0]
ss_flux_rU = -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]
print(f"ss_flux: {ss_flux_lU*C1[1,1]}")
print(f"ss_flux: {ss_flux_rU*C1[-1,-1]}")
print("FLUX = ", ss_flux_lU*C1[1,1] / 30 * 0.045 * 10**-10 / 10**-12, "mol/m^2/s")
# Below is the one in mol / m^3 / ps * A
#print(1 / (nmem * Rmem + (nwl + nwr) * nmicrobins * dz / (Dwater*Dwater_factor) ) * (X0_ff[0] - X0_ff[-1]))

# Now we increase the OCR by 300% on the right, and see what happens
t_ocr = np.linspace(0, tau_max_screen, 10000)
X0_ocr = X0_ff
Ul_ocr = np.ones_like(t_ocr)*ss_flux_lU
Ur_ocr = -1*np.ones_like(t_ocr)*ss_flux_rU
Ur_ocr -= np.array([(1.-np.exp(-t/tau_ocr)) for t in t_ocr])*ss_flux_rU*(ocr_increase-1.)

fig, ax = plt.subplots()
ax.plot(t_ocr/factor, Ul_ocr, label='Left flux')
ax.plot(t_ocr/factor, Ur_ocr, label='Right flux')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Flux (mol/m^2/s)')
ax.legend()
plt.show()

U_ocr = np.vstack((Ul_ocr,Ur_ocr))

t_out_ocr, y_out_ocr, x_out_ocr = control.forced_response(
    statespace_ff, T=t_ocr, U=U_ocr, X0=X0_ocr, return_x=True
)

# Find the index of the zero-concentration bin
nill_index = None 
for i in range(len(y_out_ocr[-1])):
    if y_out_ocr[-1][i] < 0:
        nill_index = i
        break
assert nill_index is not None
if y_out_ocr[-1][nill_index] < 0:
    nill_index -= 1
print("time to reach zero concentration:", t_out_ocr[nill_index]/factor, 'ms')
print("At nillindex", nill_index)
initial_state = x_out_ocr[:,nill_index]
initial_state[-1] = 0.
# and propagate with constant concentration right (and constant influx left)
# but we do this for longer (2 seconds, before CBF response kicks in)
tnew = np.linspace(0, tau_max, 10000)
Ul_fc = np.ones_like(tnew)*ss_flux_lU
Ur_fc = np.zeros_like(tnew)
U_fc = np.vstack((Ul_fc,Ur_fc))

t_out_fc, y_out_fc, x_out_fc = control.forced_response(
    statespace_fc, T=tnew, U=U_fc, X0=initial_state, return_x=True
)

t3 = t_out_ocr[:nill_index]/factor
t4 = t3[-1] + t_out_fc/factor
fig, ax = plt.subplots()
for i in range(1,len(x_out_fc)-1):
    ax.plot(t3, x_out_ocr[i][:nill_index]*Cchosen[i-1,i-1], '-')
    ax.plot(t4, x_out_fc[i]*Cchosen[i-1,i-1], '-')
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Oxygen concentration (A)")
fig.show()

# # we calculate the amount of oxygen consumed after the OCR increase
# # by integrating the flux over time
# # the flux is constant, so we can just multiply
# # the flux by the time
# # we integrate from the time the OCR increase starts
# # to the time it ends

# # consumed = 
# oxygens = [0.]
# time_increase = [0.]
# flux_increase = ss_flux_rU*ocr_increase*C1[-1,-1]
# for i in range(1, nill_index):
#     oxygens.append(oxygens[-1] - Ur_ocr[i]*(t_out_ocr[i]-t_out_ocr[i-1])*C1[-1,-1])
#     time_increase.append(time_increase[-1] + (t_out_ocr[i]-t_out_ocr[i-1]))

# # herafter, we set the right side at zero concentration, so we can 
# # calculate the amount of oxygen that is consumed by 
# # doing 
# for i in range(1, len(t_out_fc)):
#     flux = -1.*(x_out_fc[-1][i]-x_out_fc[-2][i])*B1[-1,-1]*C1[-1,-1]
#     oxygens.append(oxygens[-1] + flux*(t_out_fc[i]-t_out_fc[i-1]))
#     time_increase.append(time_increase[-1] + (t_out_fc[i]-t_out_fc[i-1]))

# dt = t_out_fc[2]-t_out_fc[1]
# fig, ax = plt.subplots()
# ax.plot(time_increase, oxygens, '-x', label="1mem")
# ax.set_xlabel("Time (ms)")
# ax.set_ylabel("Oxygen consumed")
# ax.legend()
# fig.show()

# flux_fc = np.array([-A_fc[i, i+1]*x_out_fc[i+1,:]*C1[i+1,i+1] + A_fc[i+1, i]*x_out_fc[i,:]*C1[i+1,i+1] for i in range(1,len(A_fc)-3)])
# flux_ocr = np.array([-A_ff[i, i+1]*x_out_ocr[i+1,:]*C1[i+1,i+1] + A_ff[i+1, i]*x_out_ocr[i,:]*C1[i+1,i+1] for i in range(1,len(A_ff)-3)])


# fig, ax = plt.subplots()

# for i in range(1,len(flux_fc)):
#     ax.plot(t3, flux_ocr[i][:nill_index], '-')
#     ax.plot(t4, flux_fc[i], '-')
# fig.show()

Target total distance: 200162.3487179487
Number of microbins left: 2797.176399531916
Number of microbins right: 245.36635083613302
Total distance: 200162.34871794868
final mids value: 200079.01538461537
ss_flux: 3.102460967433124e-05
ss_flux: 3.102460967582004e-05
FLUX =  4.653691451149686e-06 mol/m^2/s
time to reach zero concentration: 109.06090609060907 ms
At nillindex 2181


In [ ]:
# left water layer + myelin should be dz*100*76 + 15um
# right water layer should be 0.5um
dtottarget = dz*(100*76) + 200000 + 10000  # Ang
print("Target total distance:", dtottarget)
#############################################
results = []
for n_par, parameterset in enumerate(parameters):
    # Now we specify dR in the beginning
    # print("Working with parameterset: ")
    #print(parameterset)
    nmem=parameterset['nmem']
    nbins_mem=76

    dR = parameterset['dR']  # Ang
    dM = nmem * nbins_mem *dz # Ang
    dL = dtottarget - dR - dM  # Ang

    nwl = parameterset['nwl']
    nwr = parameterset['nwr']
    nmicrobinsl = dL/dz/nwl
    nmicrobinsr = dR/dz/nwr
    # print("float of microbins left:", nmicrobinsl)
    # print("float of microbins right:", nmicrobinsr)

    dtot = nwl*dz*nmicrobinsl + nwr*dz*nmicrobinsr + dM
    # print("Total distance:", dtot)

    Dwater_factor = parameterset['Dw_f']

    c_init_l = parameterset['c_init_l']
    c_init_r = parameterset['c_init_r']
    ocr_increase = parameterset['f']

    factor = 10**9
    tau_ocr = parameterset['delta_f']*factor

    A1, B1, C1, D1, mids1, K1 = get_coarse_system(dz, Dwater*Dwater_factor,
                                                nmicrobinsl, nmicrobinsr,
                                                nwl, nwr, nmem,
                                                nbins_mem)
    # print("final mids value:", mids1[-1])
    evals, evecs = np.linalg.eig(A1)
    # All eigenvalues must be negative because no sources in matrix
    tau_max = -1./np.max(evals)
    # Constant concentration left and right
    Achosen, Bchosen, Cchosen = A1, B1, C1
    Teq = np.zeros((len(Achosen)+2, len(Achosen)+2), float)
    Teq[1,0] = Bchosen[0,0]
    Teq[-2,-1] = Bchosen[-1,-1]
    for i in range(len(Achosen)):
        for j in range(len(Achosen)):
            Teq[i+1,j+1] = Achosen[i,j]
    # Constant flux left and right
    Tf = Teq.copy()
    Tf[0,0] = - Tf[1,0]
    Tf[-1,-1] = - Tf[-2,-1]
    Tf[0,1] = Tf[1,0]
    Tf[-1,-2] = Tf[-2,-1]
    # constant concentration right, constant flux left
    Tfc = Teq.copy()
    Tfc[0,0] = -Tfc[1,0]
    Tfc[0,1] = Tfc[1,0]

    # Constant flux left and right
    A_ff = Tf
    B_ff = np.zeros((len(A_ff), 2), float)
    B_ff[0,0] = B_ff[-1,-1] = 1.
    C_ff = np.eye(len(A_ff))
    D_ff = np.array([[0,0] for i in range(len(A_ff))])
    IO_FLUX = control.LinearIOSystem(
        control.StateSpace(A_ff, B_ff, C_ff, D_ff),
        inputs=2, outputs=len(A_ff), states=len(A_ff),
        name='constantqs flux left and right'
    )
    statespace_ff = control.StateSpace(A_ff, B_ff, C_ff, D_ff)
    # constant concentration right, constant flux left
    A_fc = Tfc
    B_fc = np.zeros((len(A_fc), 2), float)
    B_fc[0,0] = 1.
    B_fc[-1,-1] = 1.
    C_fc = np.eye(len(A_fc))
    D_fc = np.array([[0,0] for i in range(len(A_fc))])
    IO_FLUX_CONC = control.LinearIOSystem(
        control.StateSpace(A_fc, B_fc, C_fc, D_fc),
        inputs=2, outputs=len(A_fc), states=len(A_fc),
        name='constant flux left, cqsonstant concentration right'
    )
    statespace_fc = control.StateSpace(A_fc, B_fc, C_fc, D_fc)

    # Now we switch to the constant flux model (FF)
    # X0_ff = x_out_cc[:,-1]
    # the initial state, X0_ff, is only approximated by the final state of the CC model
    # we went 10 tau_max in the CC model, but that's not infinite time
    # we should have gone to infinity to get the exact initial state for the FF model
    # but we can calculate the exact initial state of the FF model by solving the 
    # steady-state equation for the FF model
    # It is the eigenvector of the FF model with eigenvalue 0 
    evals, evecs = np.linalg.eig(Teq)
    idx = evals.argsort()
    evals = evals[idx]
    evecs = evecs[:,idx]
    X0_ff_l = evecs[:, -2]
    X0_ff_r = evecs[:, -1]
    X0_ff = X0_ff_l/X0_ff_l[0]*c_init_l + X0_ff_r/X0_ff_r[-1]*c_init_r
    # Same thing for the steady-state flux, we can calculate it exactly
    # divisor = (dz*250/2/Dwater*dz*250)/Dwater_factor
    # U_U_ss_flux_U = -1.*(x_out_cc[1][-1]-x_out_cc[0][-1])/divisor
    # ss_flux_U = np.mean([-(X0_ff[1] - X0_ff[0])*B1[0,0],
    #                      -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]])
    #                      #weights = [C1[1,1], C1[-1,-1]])
    # print(f"SS-flux: {ss_flux_U * C1[1,1]}")

    ss_flux_lU = -(X0_ff[1] - X0_ff[0])*B1[0,0]
    ss_flux_rU = -(X0_ff[-1] - X0_ff[-2])*B1[-1, -1]
    # print("ss_flux:", ss_flux_lU*C1[1,1], ss_flux_rU*C1[-1,-1])
    # print("ss_flux_lU:", ss_flux_lU, "ss_flux_rU:", ss_flux_rU)
    parameterset['ss_flux_lU'] = ss_flux_lU
    parameterset['ss_flux_rU'] = ss_flux_rU
    parameterset['ss_flux_l'] = ss_flux_lU*C1[1,1]
    parameterset['ss_flux_r'] = ss_flux_lU*C1[-1,-1]
    # Below is the one in mol / m^3 / ps * A
    #print(1 / (nmem * Rmem + (nwl + nwr) * nmicrobins * dz / (Dwater*Dwater_factor) ) * (X0_ff[0] - X0_ff[-1]))

    # Now we increase the OCR by 300% on the right, and see what happens
    tau_max_screen = tau_max*10+tau_ocr*5
    t_ocr = np.linspace(0, tau_max_screen, 1000)
    X0_ocr = X0_ff
    Ul_ocr = np.ones_like(t_ocr)*ss_flux_lU
    Ur_ocr = -1*np.ones_like(t_ocr)*ss_flux_rU
    Ur_ocr -= np.array([(1.-np.exp(-t/tau_ocr)) for t in t_ocr])*ss_flux_rU*(ocr_increase-1.)

    U_ocr = np.vstack((Ul_ocr,Ur_ocr))

    t_out_ocr, y_out_ocr, x_out_ocr = control.forced_response(
        statespace_ff, T=t_ocr, U=U_ocr, X0=X0_ocr, return_x=True
    )

    # Find the index of the zero-concentration bin
    nill_index = None 
    for i in range(len(y_out_ocr[-1])):
        if y_out_ocr[-1][i] < 0:
            nill_index = i
            break
    assert nill_index is not None
    if y_out_ocr[-1][nill_index] < 0:
        nill_index -= 1
    print("nill_index:", nill_index, "time", t_out_ocr[nill_index]/factor, "ms")
    while nill_index < 500:
        # refine the search
        t_ocr = np.linspace(t_ocr[nill_index-1], t_ocr[nill_index+1], 1000)
        X0_ocr = x_out_ocr[:, nill_index-1]
        Ul_ocr = np.ones_like(t_ocr)*ss_flux_lU
        Ur_ocr = -1*np.ones_like(t_ocr)*ss_flux_rU
        Ur_ocr -= np.array([(1.-np.exp(-t/tau_ocr)) for t in t_ocr])*ss_flux_rU*(ocr_increase-1.)
        U_ocr = np.vstack((Ul_ocr,Ur_ocr))

        t_out_ocr, y_out_ocr, x_out_ocr = control.forced_response(
            statespace_ff, T=t_ocr, U=U_ocr, X0=X0_ocr, return_x=True
        )

        # Find the index of the zero-concentration bin
        nill_index = None 
        for i in range(len(y_out_ocr[-1])):
            if y_out_ocr[-1][i] < 0:
                nill_index = i
                break
        assert nill_index is not None
        if y_out_ocr[-1][nill_index] < 0:
            nill_index -= 1
        print(f"nill_index: {nill_index}, time: {t_out_ocr[nill_index]/factor} ms")
    parameterset['nill_index'] = nill_index
    parameterset['t_nill'] = t_out_ocr[nill_index]/factor

    print(f"Done with paramterset {n_par}")
    results.append(parameterset.copy())
    parameterset = {}